In [3]:
import pandas as pd
import numpy as np
import os
import sys
from tqdm.auto import tqdm # Import the progress bar library
from sklearn.model_selection import train_test_split
from camel_tools.disambig.bert import BERTUnfactoredDisambiguator
from camel_tools.tokenizers.word import simple_word_tokenize
from camel_tools.utils.dediac import dediac_ar

# =====================================================================================
# 1. CONFIGURATION
# =====================================================================================
BASE_DIR = '.'
DATA_DIR = os.path.join(BASE_DIR, "data")
OUTPUT_DIR = os.path.join(BASE_DIR, "colab_data")
os.makedirs(OUTPUT_DIR, exist_ok=True)

# --- Input File Paths (Raw Data) ---
BAREC_TRAIN_PATH = os.path.join(DATA_DIR, 'train.csv')
BAREC_DEV_PATH = os.path.join(DATA_DIR, 'dev.csv')
BLIND_TEST_PATH = os.path.join(DATA_DIR, 'blind_test_data.csv')
SAMER_CORPUS_PATH = os.path.join(DATA_DIR, 'samer_train.tsv')
SAMER_LEXICON_PATH = os.path.join(DATA_DIR, 'samer_lexicon.tsv')

# --- Output File Paths (Colab-Ready Data) ---
TRAIN_OUTPUT_PATH = os.path.join(OUTPUT_DIR, 'train_lex.csv')
DEV_OUTPUT_PATH = os.path.join(OUTPUT_DIR, 'dev_lex.csv')
TEST_OUTPUT_PATH = os.path.join(OUTPUT_DIR, 'test_lex.csv')

# --- Splitting & Processing Parameters ---
VALIDATION_SPLIT_SIZE = 0.15
RANDOM_STATE = 42
# BATCH_SIZE controls how many sentences are processed at once.
# A larger size is often more efficient for the CPU.
BATCH_SIZE = 128

# =====================================================================================
# 2. HELPER FUNCTION (Processes a batch of sentences)
# =====================================================================================
def process_batch(batch_texts, disambiguator, lexicon_map):
    """Processes a batch of sentences to extract D3Tok and lexical features."""
    valid_texts = [text for text in batch_texts if isinstance(text, str) and text.strip()]
    if not valid_texts:
        return [("", 0.0, 0.0)] * len(batch_texts)

    tokenized_sentences = [simple_word_tokenize(text) for text in valid_texts]
    disambiguated_batch = disambiguator.disambiguate_sentences(tokenized_sentences)
    
    results = []
    for disambiguated_sentence in disambiguated_batch:
        d3tok_forms = []
        for da in disambiguated_sentence:
            if da.analyses and 'd3tok' in da.analyses[0][1]:
                d3tok = dediac_ar(da.analyses[0][1]['d3tok']).replace("_+", " +").replace("+_", "+ ")
                d3tok_forms.append(d3tok)
            else:
                d3tok_forms.append(da.word)
        d3tok_text = " ".join(d3tok_forms)

        scores = []
        for dw in disambiguated_sentence:
            if dw.analyses:
                analysis = dw.analyses[0][1]
                lemma, pos = analysis.get('lex'), analysis.get('pos')
                if lemma and pos:
                    score = lexicon_map.get(f"{dediac_ar(lemma)}#{pos}")
                    if score is not None:
                        scores.append(score)
        
        avg_readability = np.mean(scores) if scores else 0.0
        max_readability = np.max(scores) if scores else 0.0
        results.append((d3tok_text, avg_readability, max_readability))
    return results

# =====================================================================================
# 3. MAIN SCRIPT EXECUTION
# =====================================================================================
def main():
    """Main function to perform all pre-processing in the correct order."""
    print("--- 🚀 Starting Master Pre-processing Script for Local Jupyter ---")

    # --- Step 1: Initialize Models & Load Lexicon (CPU Only) ---
    print("\n[1/5] Initializing BERT Disambiguator and loading SAMER Lexicon...")
    try:
        # Load the models normally; they will run on the CPU.
        disambiguator = BERTUnfactoredDisambiguator.pretrained('msa')
        lexicon_df = pd.read_csv(SAMER_LEXICON_PATH, sep='\t')
        lexicon_map = lexicon_df.set_index('lemma#pos')['readability (rounded average)'].to_dict()
    except Exception as e:
        print(f"❌ ERROR: Could not load required models or files: {e}")
        return
    print("✔️ Models and lexicon loaded.")

    # --- Step 2: Load and Merge Data (Unchanged) ---
    print("\n[2/5] Loading and merging all BAREC and SAMER data...")
    try:
        barec_train_df = pd.read_csv(BAREC_TRAIN_PATH)[['Sentence', 'Readability_Level_19']]
        barec_dev_df = pd.read_csv(BAREC_DEV_PATH)[['Sentence', 'Readability_Level_19']]
        samer_df = pd.read_csv(SAMER_CORPUS_PATH, sep='\t')
        samer_records = []
        for level_name, barec_level in {'L3': 4, 'L4': 10, 'L5': 16}.items():
            samer_subset = samer_df[[level_name]].dropna().rename(columns={level_name: 'Sentence'})
            samer_subset['Readability_Level_19'] = barec_level
            samer_records.append(samer_subset)
        samer_augmentation_df = pd.concat(samer_records, ignore_index=True)
        combined_df = pd.concat([barec_train_df, barec_dev_df, samer_augmentation_df], ignore_index=True)
        combined_df.dropna(inplace=True)
        combined_df = combined_df.rename(columns={'Sentence': 'text', 'Readability_Level_19': 'label'})
        print(f"✔️ Total combined records: {len(combined_df)}")
    except Exception as e:
        print(f"❌ ERROR loading source data: {e}")
        return

    # --- Step 3: Stratified Split (Unchanged) ---
    print("\n[3/5] Creating new 85%/15% stratified split...")
    final_train_df, final_dev_df = train_test_split(
        combined_df, test_size=VALIDATION_SPLIT_SIZE, random_state=RANDOM_STATE, stratify=combined_df['label']
    )
    print("✔️ Split complete.")
    print(f"   - Final Training Set Size:   {len(final_train_df)}")
    print(f"   - Final Validation Set Size: {len(final_dev_df)}")

    # --- Step 4: Process Splits with Progress Bar ---
    print("\n[4/5] Applying D3Tok and Lexicon feature extraction to new splits...")
    for name, df in [('Training', final_train_df), ('Validation', final_dev_df)]:
        print(f"  > Processing {name} data in batches...")
        all_results = []
        
        # THIS IS THE LOGGING LOOP
        # tqdm will create a progress bar showing (current_batch/total_batches)
        # For example: 13/2794 [00:10<1:30:00, 2.5 it/s]
        for i in tqdm(range(0, len(df), BATCH_SIZE), desc=f"Processing {name} Batches"):
            batch_texts = df['text'].iloc[i:i+BATCH_SIZE].tolist()
            batch_results = process_batch(batch_texts, disambiguator, lexicon_map)
            all_results.extend(batch_results)

        processed_df = pd.DataFrame(all_results, columns=['d3tok_text', 'avg_readability', 'max_readability'], index=df.index)
        df[['d3tok_text', 'avg_readability', 'max_readability']] = processed_df
        df['label'] = (df['label'].astype(int) - 1)
        
        output_path = TRAIN_OUTPUT_PATH if name == 'Training' else DEV_OUTPUT_PATH
        df[['label', 'd3tok_text', 'avg_readability', 'max_readability']].to_csv(output_path, index=False)
        print(f"✔️ Saved Colab {name} file to: {output_path}")

    # --- Step 5: Process Blind Test Set with Progress Bar ---
    print("\n[5/5] Processing the blind test set...")
    try:
        test_df = pd.read_csv(BLIND_TEST_PATH).dropna(subset=['Sentence'])
        all_test_results = []

        # Logging loop for the test set
        for i in tqdm(range(0, len(test_df), BATCH_SIZE), desc="Processing Test Batches"):
            batch_texts = test_df['Sentence'].iloc[i:i+BATCH_SIZE].tolist()
            batch_results = process_batch(batch_texts, disambiguator, lexicon_map)
            all_test_results.extend(batch_results)

        processed_test_df = pd.DataFrame(all_test_results, columns=['d3tok_text', 'avg_readability', 'max_readability'], index=test_df.index)
        test_df[['d3tok_text', 'avg_readability', 'max_readability']] = processed_test_df
        test_df[['ID', 'd3tok_text', 'avg_readability', 'max_readability']].to_csv(TEST_OUTPUT_PATH, index=False)
        print(f"✔️ Saved Colab test file to: {TEST_OUTPUT_PATH}")
    except Exception as e:
        print(f"❌ ERROR processing test data: {e}")

    print("\n--- 🎉 All files are processed and saved in the 'colab_data' folder. ---")

# In the next cell, run this to start the process:
main()

--- 🚀 Starting Master Pre-processing Script for Local Jupyter ---

[1/5] Initializing BERT Disambiguator and loading SAMER Lexicon...


Some weights of the model checkpoint at C:\Users\Fatima\AppData\Roaming\camel_tools\data\disambig_bert_unfactored\msa were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


✔️ Models and lexicon loaded.

[2/5] Loading and merging all BAREC and SAMER data...
✔️ Total combined records: 105184

[3/5] Creating new 85%/15% stratified split...
✔️ Split complete.
   - Final Training Set Size:   89406
   - Final Validation Set Size: 15778

[4/5] Applying D3Tok and Lexicon feature extraction to new splits...
  > Processing Training data in batches...


Processing Training Batches:   0%|          | 0/699 [00:00<?, ?it/s]

✔️ Saved Colab Training file to: .\colab_data\train_lex.csv
  > Processing Validation data in batches...


Processing Validation Batches:   0%|          | 0/124 [00:00<?, ?it/s]

✔️ Saved Colab Validation file to: .\colab_data\dev_lex.csv

[5/5] Processing the blind test set...


Processing Test Batches:   0%|          | 0/27 [00:00<?, ?it/s]

✔️ Saved Colab test file to: .\colab_data\test_lex.csv

--- 🎉 All files are processed and saved in the 'colab_data' folder. ---
